# BBBC021: Quality control

In [1]:
# %load ../common_.py
import sys
sys.path.append("..")
from common import *

## Data

In [2]:
data_dir = Path("/data/gent/vo/000/gvo00070/vsc42015/datasets/BBBC021/results/scip/270520220947_Week1/")

In [4]:
df = pq.read_table(data_dir / "features.parquet").to_pandas()

In [7]:
# set index